In [1]:
## Progress bar
from tqdm.notebook import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data

/home/ivanoleksiyuk/WORK/jet_clip/.env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Fetching the device that will be used throughout this notebook
device = torch.device("cpu") if not torch.cuda.is_available() else torch.device("cuda:0")
print("Using device", device)

Using device cuda:0


In [3]:
### DEFINE DATA AND HYPERPARAMETERS
data_config="QCD1"

In [4]:
### DEFINE A DATASET MANAGER
from datasets import DataManager
DM=DataManager(data_config=data_config, transform=None)

In [5]:
train_loader = data.DataLoader(
    DM.train_set, batch_size=256, shuffle=False, drop_last=False)
val_loader = data.DataLoader(
    DM.val_set, batch_size=64, shuffle=False, drop_last=False, num_workers=4)
test_loader = data.DataLoader(
    DM.test_bg, batch_size=64, shuffle=False, drop_last=False, num_workers=4)

In [6]:
class CLIPLoss(nn.Module):
    def __init__(self, logit_scale=1.):
        super().__init__()

        self.logit_scale = logit_scale

        self.__loss_evol = {'train': [], 'valid': []}

    @property
    def loss_evolution(self):
        return self.__loss_evol

    def item(self):
        return self.item_

    def forward(self, embedding_1, embedding_2, valid=False):
        device = embedding_1.device

        logits_1 = self.logit_scale * embedding_1 @ embedding_2.T
        logits_2 = self.logit_scale * embedding_2 @ embedding_1.T

        num_logits = logits_1.shape[0]
        labels = torch.arange(num_logits, device=device, dtype=torch.long)

        loss = 0.5 * (
            F.cross_entropy(logits_1, labels) +
            F.cross_entropy(logits_2, labels)
        )

        self.__loss_evol['valid' if valid else 'train'].append(loss.item())
        self.item_ = loss.item()
        return loss

In [9]:
from encodermodels import SimpleConvNetEncoder
import torch.optim as optim

clip_loss = CLIPLoss()
encoder1 = SimpleConvNetEncoder()
encoder2 = SimpleConvNetEncoder()
optimizer = optim.SGD(list(encoder1.parameters()) +
                      list(encoder2.parameters()), lr=0.001, momentum=0.9)

In [12]:
for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0

    for i, data in enumerate(train_loader):
        inputs=data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        embedding1 = encoder1(inputs)
        embedding2 = encoder2(inputs)
        loss = clip_loss(embedding1, embedding2, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += clip_loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0


TypeError: conv2d() received an invalid combination of arguments - got (list, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!list!, !Parameter!, !Parameter!, !tuple!, !tuple!, !tuple!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!list!, !Parameter!, !Parameter!, !tuple!, !tuple!, !tuple!, int)
